## Start MetaTrader 5 Docker Terminal

In [ ]:
import logging
import time
from typing import Optional
from trade_flow.common.logging import Logger
from packages.mt5any import (
    DockerizedMT5TerminalConfig,
    DockerizedMT5Terminal,
)
from packages.mt5any import MetaTrader5


class MT5Exception(Exception):
    """Base exception class for MT5 errors."""

    pass


class MT5InitializationError(MT5Exception):
    """Raised when there is an error initializing MT5 Trader."""

    pass


class MT5:
    """
    A class to interface with MetaTrader 5 for trading operations using Dockerized MT5 Terminals.

    Attributes:
        mt5_account_number (str): The MetaTrader 5 account number.
        mt5_password (str): The password for the MT5 account.
        mt5_server (str): The MT5 server name.
        logger (Logger): Logger instance for logging events.
        mt5_terminal (DockerizedMT5Terminal): Dockerized MT5 Terminal instance.
        mt5 (MetaTrader5): MetaTrader 5 client interface.
        initial_balance (float): The initial account balance.
    """

    def __init__(
        self,
        account_number: str,
        password: str,
        server: str,
        logger: Optional[Logger] = None,
    ) -> None:
        """
        Initialize the MT5 class with account credentials and configurations.

        Args:
            account_number (str): MetaTrader 5 account number.
            password (str): MetaTrader 5 account password.
            server (str): MetaTrader 5 server.
            logger (Optional[Logger]): Logger instance for logging.
        """
        self.mt5_account_number = account_number
        self.mt5_password = password
        self.mt5_server = server

        # Set up logging
        self.logger = logger or Logger(name="tft_logger", log_level=logging.DEBUG, filename="TFT_model_experiments.log")

        # Set up MetaTrader 5 terminal and configuration
        self.mt5_config = DockerizedMT5TerminalConfig(
            account_number=self.mt5_account_number,
            password=self.mt5_password,
            server=self.mt5_server,
            read_only_api=True,
        )

        # Initialize Dockerized MT5 Terminal
        self.mt5_terminal = DockerizedMT5Terminal(config=self.mt5_config)
        self._initialize_terminal()

        # Initialize MetaTrader 5
        self.mt5 = MetaTrader5()
        self._initialize_mt5()
        self.logger.debug(f"Terminal Info: {self.mt5.terminal_info()._asdict()}")

        # Get account information
        self.account_info = self.mt5.account_info()._asdict()
        self.logger.debug(f"Account Info: {self.account_info}")

        self.initial_balance = self.account_info["balance"]

        # Log account info
        self.logger.info(f"Account Balance: {self.initial_balance}")
        self.logger.info(f"Equity: {self.account_info['equity']}")
        self.logger.info(f"Currency: {self.account_info['currency']}")
        self.logger.info(f"Margin: {self.account_info['margin']}")
        self.logger.info(f"Server: {self.account_info['server']}")
        self.logger.info(f"Name: {self.account_info['name']}")

    def _initialize_terminal(self) -> None:
        """Initialize and safely start the Dockerized MT5 Terminal."""
        try:
            self.mt5_terminal.safe_start()
            time.sleep(5)

            self.logger.info(f"MetaTrader 5 Terminal started for account {self.mt5_account_number}")
        except Exception as e:
            self.logger.error(f"Error initializing Dockerized MT5 Terminal: {e}")
            raise MT5InitializationError("Failed to start Dockerized MT5 Terminal")

    def _initialize_mt5(self) -> None:
        """Initialize the MetaTrader 5 client."""
        try:
            if not self.mt5.initialize():
                raise RuntimeError("MetaTrader 5 initialization failed")

            # if not self.mt5.login(self.mt5_account_number, self.mt5_password, self.mt5_server):
            #     raise RuntimeError("MetaTrader 5 login failed")
        except Exception as e:
            self.logger.error(f"Error initializing MetaTrader 5: {e}")
            raise MT5InitializationError("Failed to initialize MetaTrader 5")


/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/trade_flow/feed/__init__.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, parse_dates=True, index_col=index_name)


In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

mt5_venue = MT5(
    account_number=os.getenv("MT5_ACCOUNT_NUMBER"),
    password=os.getenv("MT5_PASSWORD"),
    server=os.getenv("MT5_SERVER"),
)


[TradeFlow] 35308 - 2024-10-18, 09:04:48 AM   [INFO] DockerizedMT5Terminal: Ensuring terminal is running
[TradeFlow] 35308 - 2024-10-18, 09:04:48 AM   [INFO] DockerizedMT5Terminal: status=<ContainerStatus.READY: 6>, using existing container
[TradeFlow] 35308 - 2024-10-18, 09:04:53 AM   [INFO] tft_logger: MetaTrader 5 Terminal started for account 30565290
[TradeFlow] 35308 - 2024-10-18, 09:04:53 AM   [DEBUG] tft_logger: Terminal Info: {'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': True, 'build': 4490, 'maxbars': 100000, 'codepage': 0, 'ping_last': 131835, 'community_balance': 0.0, 'retransmission': 0.0018427405237068566, 'company': 'MetaQuotes Ltd.', 'name': 'MetaTrader 5', 'language': 'English', 'path': 'C:\\Program Files\\MetaTrader 5', 'data_path': 'C:\\Program Files\\MetaTrader 5', 'commondata_

In [ ]:
mt5_venue

## Imports

In [ ]:
from typing import List, Tuple
from datetime import datetime
from trade_flow.environments import metatrader
from trade_flow.environments.metatrader import Timeframe, FOREX_DATA_PATH, STOCKS_DATA_PATH, CRYPTO_DATA_PATH, MIXED_DATA_PATH

/home/fortesenselabs/Tech/labs/Financial_Eng/Financial_Markets/lab/trade_flow/trade_flow/feed/__init__.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(path, parse_dates=True, index_col=index_name)


In [2]:
def get_data(symbols: List[str] = ["EURUSD", "GBPCAD", "USDJPY"], 
             time_range: Tuple[datetime, datetime] = (datetime(2011, 1, 1), datetime(2012, 12, 31)),
             timeframe: Timeframe = Timeframe.D1, 
             filename: str = FOREX_DATA_PATH):
    
    mt_sim = metatrader.Simulator()
    mt_sim.download_data(symbols, time_range, timeframe)
    mt_sim.save_symbols(filename)


In [3]:
# Define the time range for the data download
start_date = datetime(2016, 1, 1)
end_date = datetime(2020, 12, 31)
time_range = (start_date, end_date)

In [4]:
# forex
get_data(["EURUSD", "GBPCAD", "USDJPY"], time_range, Timeframe.D1, FOREX_DATA_PATH)

In [5]:
# stocks
get_data(["GOOG", "AAPL", "TSLA", "MSFT"], time_range, Timeframe.D1, STOCKS_DATA_PATH)

In [6]:
# crypto
get_data(["BTCUSD", "ETHUSD", "BCHUSD"], time_range, Timeframe.D1, CRYPTO_DATA_PATH)

In [7]:
# mixed 
get_data(["EURUSD", "USDCAD", "GOOG", "AAPL", "BTCUSD", "ETHUSD"], time_range, Timeframe.D1, MIXED_DATA_PATH)